# Breast Cancer Classification: ResNet50
This notebook implements the Transfer Learning model for breast cancer classification.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, MaxPooling2D, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

sys.path.append('..')
from report_generator import ReportGenerator

In [ ]:
DATASET_ROOT = '../dataset'
OUTPUT_DIR = 'output_resnet50'
os.makedirs(OUTPUT_DIR, exist_ok=True)

def load_data(data_dir):
    images, labels = [], []
    for ds in ['Augmented Dataset', 'Original Dataset']:
        path = os.path.join(data_dir, ds)
        if not os.path.exists(path): path = data_dir
        for label in ['Cancer', 'Non-Cancer']:
            l_path = os.path.join(path, label)
            if os.path.isdir(l_path):
                for f in os.listdir(l_path):
                    if f.lower().endswith(('.png','.jpg','.jpeg')):
                        img = cv2.imread(os.path.join(l_path, f), cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img = cv2.resize(img, (128,128))
                            img = cv2.equalizeHist(img)
                            images.append(img/255.0)
                            labels.append(1 if label=='Cancer' else 0)
    return np.array(images), np.array(labels)

def apply_clahe(images):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    processed = [clahe.apply((img*255).astype(np.uint8))/255.0 for img in images]
    return np.array(processed)

images, labels = load_data(DATASET_ROOT)
X = apply_clahe(images).reshape(-1,128,128,1)
y = to_categorical(labels, 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.applications import ResNet50
def build_model():
    base = ResNet50(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
    for layer in base.layers[:-4]: layer.trainable = False
    model = Sequential([
        Input(shape=(128, 128, 1)),
        Conv2D(3, (3, 3), padding='same', activation='relu', name='channel_conversion'),
        base, GlobalAveragePooling2D(),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)), Dropout(0.4),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    return model, base

model, base_model = build_model()
history = model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))